In [ ]:
from pylab import *
import csv
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot
import numpy as np
from tabulate import tabulate
import cycle_preparation as cp

In [ ]:
def calculate_braking(cycle_opened, limit_natural):
    """
    Calculate average braking force (B) and maximum brajking force (B_max) of the provided cycle or target speed.

    Parameters:
    cycle_opened (tuple): Contains cycle data with list_D and extrema values of the driving cycle.
    limit_natural (float): The limit for the braking force to become a natural deceleration (i.e. braking is neglected in that context).

    Returns:
    tuple: Rounded values of limit natural, weighted average braking force (B), and maximum braking force (B_max).
    """
    list_D = cycle_opened[1] #the cycle is now the speed as a function of the distance
    ext = cycle_opened[2]

    # Identify incident positions where deceleration occurs
    values = np.array(ext[0])
    diff_values = np.diff(values)
    incident_positions = np.where(diff_values < 0)[0]
    
    
    # Calculate variation of kinetic energy during the braking
    kinetic_energy_values = values ** 2 / 2
    energy_diff = np.diff(kinetic_energy_values)

    # Calculate braking distances and forces
    list_incident_D = list_D[ext[1]]
    diff_pos = np.diff(list_incident_D)
    braking_distances = diff_pos[incident_positions]
    braking_forces = -energy_diff[energy_diff < 0] / braking_distances

    # Calculate weighted average and maximum braking force
    valid_forces = braking_forces[braking_forces > limit_natural]
    weighted_avg_braking_force = np.average(valid_forces, weights=-energy_diff[energy_diff < 0][braking_forces > limit_natural])
    max_braking_force = np.max(braking_forces)

    # Round the results
    limit_natural = np.round(limit_natural, 2)
    B = np.round(weighted_avg_braking_force, 2)
    B_max = np.round(max_braking_force, 2)

    return limit_natural, B, B_max


In [ ]:
def calculate_acceleration(cycle_opened):
    """
    Calculate the average power per mass Pa[W/kg] applied to the automobile during acceleration phases in the provided cycle or target speed.

    Parameters:
    cycle_opened (tuple): Contains cycle data with list_D and extrema values.

    Returns:
    float: Rounded average acceleration power per mass (Pa).
    """
    list_D = cycle_opened[1]
    ext = cycle_opened[2]

    # Identify positions where acceleration occurs
    values = np.array(ext[0])
    diff_values = np.diff(values)
    acceleration_positions = np.where(diff_values > 0)[0]
    

    # Calculate variation of kinetic energy
    values_2 = values ** 2 / 2
    values_2_pos = np.diff(values_2)

    # Calculate acceleration durations and power per mass
    list_incident = ext[1]
    diff_duration = np.diff(list_incident)
    acceleration_duration = diff_duration[acceleration_positions]
    acceleration_power_permass = values_2_pos[values_2_pos > 0] / acceleration_duration

    # Calculate the weighted average of acceleration power per mass
    Pa = np.average(acceleration_power_permass, weights=values_2_pos[values_2_pos > 0])

    # Round the result
    Pa = np.round(Pa, 2)

    return Pa


In [ ]:
def type_road(path):
    """
    Classify road types based on speed limits.

    Parameters:
    path (array): driving cycle (Array of speed).

    Returns:
    tuple: 
        - road (array): Array indicating road types (0: unknown, 1: city, 2: suburban road, 3: countryside, 4: highway).
        - indices (list): List of indices for each road type.
    """
    road=np.zeros((len(path)))
    road[np.where(path>0)]=1 #city roads are defined as all roads with speed limitation < 55 km/h
    road[np.where(path>55/3.6)]=2  #suburban roads are defined as all roads with speed limitation between 55-75 km/h
    road[np.where(path>75/3.6)]=3 #contryside roads are defined as all roads with speed limitation between 75-100 km/h
    road[np.where(path>100/3.6)]=4 #highways driving is defined as all roads with speed limitation >100 km/h
    return road,[np.where(road==0),np.where(road==1),np.where(road==2),np.where(road==3),np.where(road==4)]
    

In [ ]:
def path_integration(cycle_opened_smooth):
    """
    Calculate dynamic variables integrals evaluated from the path target speed.
    These parameters characterized the path solely and are independent of the driver.
    Global path parameters are calculated to characterize the whole journey.
    The path is further analyzed and divided into subcycles depending on the type of road (city, suburban, countryside, and highway). 
    Additionally, a detailed evaluation of the parameters for these subcycles is performed.

    Parameters:
    cycle_opened_smooth (tuple): Contains path and cycle data.

    Returns:
    list: the path parameters.
    """
    # Initialisation
    list_D = cycle_opened_smooth[1]
    dist_tot = list_D[-1]
    diff_pos = np.concatenate(([0], np.diff(list_D)))

    ext = cycle_opened_smooth[2]
    values = np.array(ext[0])
    diff_values = np.diff(values)
    list_incident_D = list_D[ext[1]]

    # Preparation for calculating K1 and K2

    values_2 = values ** 2 / 2
    values_2_pos = np.diff(values_2)
    values_3 = values ** 3 / 3
    values_3_pos = np.diff(values_3)

    # Calculate braking forces
    diff_pos_incident = np.diff(list_incident_D)
    incident_pos = np.where(diff_values < 0)[0]
    braking_distance = diff_pos_incident[incident_pos]
    braking_force = -values_2_pos[values_2_pos < 0] / braking_distance

    # determining the type of road along the journey
    path = cycle_opened_smooth[3]
    path_type = type_road(path)

    results = []
    urban_proportion = 0 #r_urban

    ### Calculate path parameters per road type first ###
    
    for k in range(1, 5): 
        
        # the driving cycle is reduce to the moments when the path is following the type of road k.
        indices = np.where(path_type[0] == k)
        dist_type = np.sum(diff_pos[indices])

        if dist_type == 0:
            results.append([nan] * 7)
            continue

        
        proportion = round(dist_type / dist_tot * 100, 1)
        
        if k < 3:
            urban_proportion += proportion #urban corresponds to city + suburban roads

        extrema_considered = ext[1][:-1]
        incident = np.where(np.isin(extrema_considered, indices))[0]

        values_2_incident = values_2_pos[incident]
        values_3_incident = values_3_pos[incident]

        braking_distance_incident = diff_pos_incident[incident]
        braking_force_incident = -values_2_incident[values_2_incident < 0] / braking_distance_incident[values_2_incident < 0]

        # path parameters evaluation for the driving cycle when we are on the road type k.
        J3_path = np.sum(path[indices] ** 2 * diff_pos[indices]) / dist_type
        J0_path = np.sum(diff_pos[indices] / path[indices]) / dist_type
        K1 = np.sum(-values_2_incident[values_2_incident < 0]) / dist_type
        K2 = np.sum(-values_3_incident[values_3_incident < 0]) / dist_type

        # for countryside and highway roads, the incidents related to exiting the road are neglected as their occurrence is too high in driving cycle.
        diff_incident = np.concatenate((np.diff(incident), [len(ext[0]) - incident[-1]]))
        non_exit_incidents = incident[diff_incident == 1]

        values_2_non_exit = values_2_pos[non_exit_incidents]
        K1_non_exit = np.sum(-values_2_non_exit[values_2_non_exit < 0]) / dist_type

        braking_distance_non_exit = diff_pos_incident[non_exit_incidents]
        braking_force_non_exit = -values_2_non_exit[values_2_non_exit < 0] / braking_distance_non_exit[values_2_non_exit < 0]

        # Round and store results
        J3_path = round(J3_path)
        J0_path = round(J0_path, 3)
        K1 = round(K1, 3)
        K1_non_exit = round(K1_non_exit, 3)
        K2 = round(K2, 3)
        dist_type = round(dist_type / 1000, 2)

        if k <= 2:
            results.append([proportion, J3_path, K1, nan, K2, nan, J0_path])
        else:
            results.append([proportion, J3_path, K1, K1_non_exit, K2, nan, J0_path])

    ### Calculate the global path parameters for the driving cycle ###
    
    J3_path = np.sum(path ** 2 * diff_pos) / dist_tot
    J0_path = np.sum(diff_pos[path > 0] / path[path > 0]) / dist_tot
    K1 = np.sum(-values_2_pos[values_2_pos < 0]) / dist_tot
    K2 = np.sum(-values_3_pos[values_3_pos < 0]) / dist_tot
    

    incident_moy = np.sum((values_2[:-1] * values_2_pos)[values_2_pos > 0]) / (K1 * dist_tot)
    top_moy = np.sum((values_2[1:] * values_2_pos)[values_2_pos > 0]) / (K1 * dist_tot)

    rate_acc = np.sqrt((incident_moy + top_moy) / (2 * top_moy))

    # Round final results
    J3_path = round(J3_path)
    J0_path = round(J0_path, 3)
    K1 = round(K1, 3)
    K2 = round(K2, 3)
    rate_acc = round(rate_acc, 3)
    dist_tot = round(dist_tot / 1000, 2)

    results.append([dist_tot, J3_path, K1, K2, rate_acc, J0_path, urban_proportion])

    return results


In [ ]:
"""
    Generation of the path parameters for the main driving cycles found in the literature.
    For each driving cycle, the path target speed is generated. 
    The global path parameters are given. 
    Additionally, 4 tabs (one per type of road) are given to characterize the path parameters on these types of road.
"""

# Read the cycle configuration file
df = pd.read_csv('cycle_for_config.csv', sep=';')
cycle_names = df.iloc[:, 0].values

# Initialize variables
results = []
limit_natural = 0.2

# Loop through each cycle file and calculate results
for cycle_name in cycle_names:
    file_name = cycle_name + ".csv"
    print(f"Processing: {file_name}")
    
    # Prepare the target function for analyse of braking and acceleration
    name_opened = cp.cycle_prepared(file_name)
    
    # Perform braking and acceleration calculations
    limit_natural, braking_force, max_braking_force = calculate_braking(name_opened, limit_natural)
    Pa = calculate_acceleration(name_opened)

    # Extract idle time and distance
    idle_time = name_opened[-1]
    total_distance = name_opened[1][-1]

    # smooth the driving cycle with the two rules (see cycle_preparation.py) and generate the target function 
    name_opened_smooth = cp.cycle_prepared(file_name, limit_natural)
    
    #calculate path variables
    results_path = path_integration(name_opened_smooth)

    # Concatenate results
    combined_results = np.concatenate(
        (results_path, [[idle_time / total_distance]] * 5, [[braking_force, max_braking_force, Pa]] * 5),
        axis=1
    )
    results.append(combined_results)

# Define column names for the result DataFrames
general_columns = ['distance', 'J3_path', 'K1', 'K2', 'rate_acc', 'J0_path', 'urban', 't_idle', 'B_tot', 'Bmax', 'Pa']
type_columns = ['proportion', 'J3_path', 'K1', 'K1_ne', 'K2', 'rate_acc', 'J0_path', 't_idle', 'B_tot', 'Bmax', 'Pa']

# Convert results to a numpy array and create DataFrames
results = np.array(results)
bilan_all = pd.DataFrame(results[:, 4, :], index=cycle_names, columns=general_columns)
bilan_city = pd.DataFrame(results[:, 0, :], index=cycle_names, columns=type_columns)
bilan_sub = pd.DataFrame(results[:, 1, :], index=cycle_names, columns=type_columns)
bilan_count = pd.DataFrame(results[:, 2, :], index=cycle_names, columns=type_columns)
bilan_hwy = pd.DataFrame(results[:, 3, :], index=cycle_names, columns=type_columns)

# Save results to an Excel file
output_file_name = 'Results.xlsx'

with pd.ExcelWriter(output_file_name) as writer:
    bilan_all.to_excel(writer, sheet_name='General', index=True)
    bilan_city.to_excel(writer, sheet_name='City', index=True)
    bilan_sub.to_excel(writer, sheet_name='Suburb', index=True)
    bilan_count.to_excel(writer, sheet_name='Countryside', index=True)
    bilan_hwy.to_excel(writer, sheet_name='Highway', index=True)

print('Processing complete. Results saved to:', output_file_name)
